# MATH50003 Numerical Analysis: Problem 3

This problem sheet explores implementation of triangular solves,
supporting a matrix with two super-diagonals, as well as
permutation and Householder reflections that can be applied to a vector in
$O(n)$ complexity.

In [1]:
using LinearAlgebra, Test

# We will override these functions below
import Base: getindex, setindex!, size, *, \

## 1. Dense Matrices

**Problem 1.1** Show that `A*x` is not
implemented as `mul(A, x)` from the lecture notes
by finding a `Float64` example  where the bits do not match.

In [2]:
# Question 1.1

 function mul(A, x)
    m, n = size(A)
    c = zeros(eltype(x), m)
    for j = 1:n, k = 1:m
        c[k] += x[j] * A[k, j]
    end
    c
end

mul (generic function with 1 method)

In [3]:
A = [1 2 3;
     4 5 6;
     7 8 9]

x = [1, 2, 3]

println(mul(A,x))
print(A*x)

[14, 32, 50]
[14, 32, 50]

In [4]:
# Find example

function findex(n, l)
    for j = 1:n
        A = randn(l,l)
        x = rand(l)
        if A*x != mul(A, x)
            return (A,x)
        end
    end
end

n = 100  # number of attempts
l = 10  # dimensions

(A,x) = findex(n, l)

norm(A*x - mul(A, x))

3.2957919968262575e-16

## 2. Triangular Matrices

**Problem 2.1** Complete the following functions for solving linear systems with
triangular systems by implementing back and forward-substitution:

In [5]:
mat = [1 2 3
       4 5 6]
size(mat, 2)

3

In [6]:
function udiv(U::UpperTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n) 
    x[n] = b[n]/U[n, n]    
    for i in n-1:-1:1
        S = 0
        for j in i+1:n
            S += U[i, j] * x[j]
        end
        x[i] = (b[i] - S)/U[i, i]
    end
    x
end

function ldiv(U::LowerTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  
    x[1] = b[1]/U[1,1]
    for i in 2:n
        S = 0
        for j in 1:i-1
            S += U[i, j] * x[j]
        end
        x[i] = (b[i] - S)/U[i, i]
    end
    x
end

ldiv (generic function with 1 method)

In [7]:
# Checking
A = randn(3,3)
b = randn(3)
A = UpperTriangular(A)
println(A\b)

B = LowerTriangular(A.data)
print(B\b)

[16.605894709326172, 9.013965113462055, -0.4398644479870723]
[7.240792300046777, 9.686490843927656, -3.3599233444086853]

In [8]:
println(udiv(A, b))
print(ldiv(B, b))

[16.605894709326176, 9.013965113462055, -0.4398644479870723]
[7.240792300046777, 9.686490843927656, -3.359923344408684]

**Problem 2.2⋆** Given $𝐱 ∈ ℝ^n$, find a lower triangular matrix of the form
$$
L = I - 2 𝐯 𝐞_1^⊤
$$
such that:
$$
L 𝐱 = x_1 𝐞_1.
$$
What does $L𝐲$ equal if $𝐲  ∈ ℝ^n$ satisfies $y_1 = 𝐞_1^⊤ 𝐲 = 0$?

In [9]:
# Question 2.2

# Lx = (I - 2ve₁ᵀ)x
#    = x - 2ve₁ᵀx
#    = x - 2vx₁
# soooooo
# I-2ve₁ᵀ = 
# 1 0 0   0 0       2v₁ 0 0   0 0
# 0 1 0   0 0       2v₂ 0 0   0 0
# 0 0 1   0 0       2v₃ 0 0   0 0
#               -   
# 0 0 0   1 0       
# 0 0 0   0 1       2vₙ 0 0   0 0
# so choose v such that v₁=0 and vₖ = xₖ/2x₁ for x₁≠0, else...

# If y₁ = 0, then Ly = y :)

## 3. Banded matrices

**Problem 3.1** Complete the implementation of `UpperTridiagonal` which represents a banded matrix with
bandwidths $(l,u) = (0,2)$:

In [10]:
# BANDWIDTH = ie the nunmber of steps from the diagonal so that all entries are 0
# so (0,2) means all below diag are 0 and all above 2nd upper diag are 0

In [11]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    # TODO: return U[k,j]

    if k==j
        return U.d[k]
    elseif k+1==j
        return U.du[k]
    elseif k+2==j
        return U.du2[k]
    else
        return zero(eltype(U))
    end
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if (j > k+2) || (j<k)
        error("Cannot modify off-band")
    end

    # TODO: modify d,du,du2 so that U[k,j] == v

    if k==j
        U.d[k] = v
    elseif k+1==j
        U.du[k] = v
    elseif k+2==j
        U.du2[k] = v
    end

    U # by convention we return the matrix
end

setindex! (generic function with 91 methods)

In [12]:
A = UpperTridiagonal([1,2,3,4], [1,2,3], [1,2])

4×4 UpperTridiagonal{Int64}:
 1  1  1  0
 0  2  2  2
 0  0  3  3
 0  0  0  4

In [13]:
A[1,1] = 5
A

4×4 UpperTridiagonal{Int64}:
 5  1  1  0
 0  2  2  2
 0  0  3  3
 0  0  0  4

**Problem 3.2** Complete the following implementations of `*` and `\` for `UpperTridiagonal` so that
they take only $O(n)$ operations.

In [14]:
function *(U::UpperTridiagonal, x::AbstractVector)
    T = promote_type(eltype(U), eltype(x)) # make a type that contains both the element type of U and x
    b = zeros(T, size(U,1)) # returned vector
    # TODO: populate b so that U*x == b (up to rounding)

    for i in 1:(size(U,1)-2)
        b[i] = U.d[i] * x[i] + U.du[i] * x[i+1] + U.du2[i] * x[i+2]
    end

    b[end-1] = U.d[end-1] * x[end-1] + U.du[end] * x[end]
    b[end] = U.d[end] * x[end]

    b
end

function \(U::UpperTridiagonal, b::AbstractVector)
    T = promote_type(eltype(U), eltype(b)) # make a type that contains both the element type of U and b
    x = zeros(T, size(U,2)) # returned vector
    m, n = size(U)
    # TODO: populate x so that U*x == b (up to rounding)

    if size(U, 1) != length(b)
        error("System not compatible.")
    end

    x[n] = b[n]/U[m, n]
    x[n-1] = (b[n-1]- x[n]*U[m-1, n])/U[m-1, n-1]

    for i in 2:n-1
        x[n-i] = (b[n-i] - dot(U[m-i, n-i+1:n-i+2], x[n-i+1:n-i+2]))/U[m-i, n-i]
    end

    x
end

\ (generic function with 136 methods)

In [15]:
Abanded = UpperTridiagonal([1.1,2.2,3.3,4.4], [1.9,2.8,3.7], [1.5,2.4])
Adense = Matrix(Abanded) # one of many easy ways to convert to dense storage

Adense == Abanded

true

In [16]:
x = [5.2,3/4,2/3,9.1415]
Adense*x

4-element Vector{Float64}:
  8.145
 25.456266666666668
 36.02355000000001
 40.22260000000001

In [17]:
Abanded*x

4-element Vector{Float64}:
  8.145
 25.456266666666668
 36.02355000000001
 40.22260000000001

In [18]:
x = [5.2,3/4,2/3,9.1415]
Adense\x

4-element Vector{Float64}:
  6.277487146066222
  0.7820538024876873
 -2.1274253902663
  2.0776136363636364

In [19]:
Abanded\x

4-element Vector{Float64}:
  6.277487146066222
  0.7820538024876865
 -2.1274253902662994
  2.0776136363636364

## 4. Permutations

**Problem 4.1⋆** What are the permutation matrices corresponding to the following permutations?
$$
\begin{pmatrix}
1 & 2 & 3 \\
3 & 2 & 1
\end{pmatrix}, \begin{pmatrix}
1 & 2 & 3 & 4 & 5 & 6\\
2 & 1 & 4 & 3 & 6 & 5
\end{pmatrix}.
$$

In [ ]:
# 0 0 1
# 0 1 0
# 1 0 0

# 0 1 0 0 0 0
# 1 0 0 0 0 0
# 0 0 0 1 0 0
# 0 0 1 0 0 0
# 0 0 0 0 0 1
# 0 0 0 0 1 0

**Problem 4.2** Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` and such that `*` takes $O(n)$ operations.

In [26]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

size(P::PermutationMatrix) = (length(P.p),length(P.p))

function getindex(P::PermutationMatrix, k::Int, j::Int)
    # TODO: Return P[k,j]
    if P.p[k] == j
        return 1
    else
        return 0
    end
end

function *(P::PermutationMatrix, x::AbstractVector)
    # TODO: permute the entries of x
    x[P.p]
end

# If your code is correct, this "unit test" will succeed
p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]


Test Passed
  Expression: P == (I(5))[p, :]
   Evaluated: [1 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 1; 0 0 … 0 0] == sparse([1, 3, 5, 2, 4], [1, 2, 3, 4, 5], Bool[1, 1, 1, 1, 1], 5, 5)

In [39]:
a = [1,2,3,4,5]
a[[5,4,3,2,1]]

5-element Vector{Int64}:
 5
 4
 3
 2
 1

In [41]:
using BenchmarkTools, Random
x=0 # for some reason, an error "UndefVarError: x not defined" will occur without this line. p and P are previously defined so they work fine.
for n = 10 .^ (1:7)
    print("n = ", n)
    p = randperm(n)
    P = PermutationMatrix(p)
    x = randn(n)
    @btime P*x
end

n = 10  110.098 ns (1 allocation: 144 bytes)
n = 100  215.769 ns (1 allocation: 896 bytes)
n = 1000  1.280 μs (1 allocation: 7.94 KiB)
n = 10000  15.300 μs (2 allocations: 78.17 KiB)
n = 100000  249.200 μs (2 allocations: 781.30 KiB)
n = 1000000  6.398 ms (2 allocations: 7.63 MiB)
n = 10000000  185.351 ms (2 allocations: 76.29 MiB)


## 5. Orthogonal matrices

**Problem 5.1⋆** Show that orthogonal matrices preserve the 2-norm of vectors:
$$
\|Q 𝐯\| = \|𝐯\|.
$$


**Problem 5.2⋆** Show that the eigenvalues $λ$ of an orthogonal matrix $Q$ are
on the unit circle: $|λ| = 1$.


**Problem 5.3⋆** Explain why an orthogonal matrix $Q$ must be equal to $I$ if all its eigenvalues are 1.


**Problem 5.4** Complete the implementation of a type representing
reflections that supports `Q[k,j]` and such that `*` takes $O(n)$ operations.

In [ ]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

size(Q::Reflection) = (length(Q.v),length(Q.v))

function getindex(Q::Reflection, k::Int, j::Int)
    # TODO: Return Q[k,j]
end

function *(P::Reflection, x::AbstractVector)
    # TODO: permute the entries of x
end

# If your code is correct, these "unit tests" will succeed
x = randn(5)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q*v ≈ -v
@test Q'Q ≈ I

**Problem 5.5** Complete the following implementation of a Housholder reflection  so that the
unit tests pass.

In [ ]:
function householderreflection(s::Bool, x::AbstractVector)
    # TODO: implement Householder reflection, returning a Reflection
end

x = randn(5)
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ≈ norm(x)*[1; zeros(length(x)-1)]

Q̃ = householderreflection(false, x)
@test Q̃ isa Reflection
@test Q̃*x ≈ -norm(x)*[1; zeros(length(x)-1)]